# 第9章 时序数据

In [2]:
import pandas as pd
import numpy as np

## 一、时序的创建
###  1. 四类时间变量
#### 现在理解可能关于③和④有些困惑，后面会作出一些说明

名称 | 描述 | 元素类型 | 创建方式  
:-|:-|:-|:-
① Date times（时间点/时刻） | 描述特定日期或时间点 | Timestamp | to_datetime或date_range
② Time spans（时间段/时期） | 由时间点定义的一段时期 | Period | Period或period_range
③ Date offsets（相对时间差） | 一段时间的相对大小（与夏/冬令时无关） | DateOffset | DateOffset
④ Time deltas（绝对时间差） | 一段时间的绝对大小（与夏/冬令时有关） | Timedelta | to_timedelta或timedelta_range

## 五、问题与练习
#### 【问题一】 如何对date_range进行批量加帧操作或对某一时间段加大时间戳密度？
在指定时间区间，调节periods（时间点个数）或者freq（间隔方法）
#### 【问题二】 如何批量增加TimeStamp的精度？
题目不理解，看似与问题一一致
#### 【问题三】 对于超出处理时间的时间点，是否真的完全没有处理方法？
将此类时间点整体平移到范围内某一年处理。（不包括尾数为00的年份适用于“28年后可以重复使用”）
#### 【问题四】 给定一组非连续的日期，怎么快速找出位于其最大日期和最小日期之间，且没有出现在该组日期中的日期？
pd.bdate_range(start=datemin,end=datemax,freq='D') - datelist，见练习二（C）。


#### 【练习一】 现有一份关于某超市牛奶销售额的时间序列数据，请完成下列问题：
#### （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）
#### （b）计算除去春节、国庆、五一节假日的月度销售总额
#### （c）按季度计算周末（周六和周日）的销量总额
#### （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和
#### （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

In [11]:
# （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）
df = pd.read_csv('data/time_series_one.csv')
print('星期',str(pd.to_datetime(df.loc[df['销售额'].idxmax(),'日期']).dayofweek+1))


星期 7


In [12]:
# （b）计算除去春节、国庆、五一节假日的月度销售总额 
df['日期'] = pd.to_datetime(df['日期'])
df_new = df.set_index('日期',drop=True)

weekmask = 'Sun Mon Tue Wed Thu Fri Sat'
holidays = pd.to_datetime([
    '2017-4-29','2017-4-30','2017-5-1',
    '2017-10-1','2017-10-2','2017-10-3','2017-10-4','2017-10-5','2017-10-6','2017-10-7','2017-10-8',
    '2018-2-15','2018-2-16','2018-2-17','2018-2-18','2018-2-19','2018-2-20','2018-2-21',
    '2018-4-29','2018-4-30','2018-5-1',
    '2018-10-1','2018-10-2','2018-10-3','2018-10-4','2018-10-5','2018-10-6','2018-10-7',
    '2019-2-4' ,'2019-2-5' ,'2019-2-6' ,'2019-2-7' ,'2019-2-8' ,'2019-2-9' ,'2019-2-10',
    '2019-5-1' ,'2019-5-2' ,'2019-5-3' ,'2019-5-4' ,
    '2019-10-1','2019-10-2','2019-10-3','2019-10-4','2019-10-5','2019-10-6','2019-10-7',
                          ])
df_new_w = df_new.loc[pd.bdate_range(start='2017-2-17',end='2019-11-13',freq='C',weekmask = weekmask,holidays=holidays)]
df_new_w.resample('M').sum().head()


,销售额
2017-02-28,31740
2017-03-31,80000
2017-04-30,70427
2017-05-31,81262
2017-06-30,80750


In [13]:
# （c）按季度计算周末（周六和周日）的销量总额
weekmask = 'Sun Sat'
df_new_eq = df_new.loc[pd.bdate_range(start='2017-2-17',end='2019-11-13',freq='C',weekmask = weekmask)]
df_new_eq.resample('Q').sum().head()

,销售额
2017-03-31,32894
2017-06-30,66692
2017-09-30,69099
2017-12-31,70384
2018-03-31,74671


In [15]:
# （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和

df_new_5d = df_new.loc[pd.bdate_range(start='2017-2-17',end='2019-11-13',freq='C')]
df_new_5d.rolling(window=5,min_periods=-5).sum().iloc[df_new_5d.shape[0]%5-1::5]

,销售额
2017-02-22,9855.0
2017-03-01,12296.0
2017-03-08,13323.0
2017-03-15,13845.0
2017-03-22,11356.0
...,...
2019-10-16,15874.0
2019-10-23,13843.0
2019-10-30,16202.0
2019-11-06,13506.0


In [23]:
# （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）
df_2018_1=df_new.loc[pd.bdate_range(start='2018-1-1',end='2018-12-31',freq='C',weekmask = 'Mon')]

for i in range(12):
    date = df_2018_1['2018-{}'.format(i+1)].index[0]   #找到每个月第一个星期一
    sale = df_new.loc[date+pd.offsets.Day(4)].values[0]#使用星期五的销售额
    print(date.date(),sale)
    

2018-01-01 2250
2018-02-05 2531
2018-03-05 2940
2018-04-02 2351
2018-05-07 2314
2018-06-04 3646
2018-07-02 2367
2018-08-06 3200
2018-09-03 2629
2018-10-01 2447
2018-11-05 2523
2018-12-03 2950


#### 【练习二】 继续使用上一题的数据，请完成下列问题：
#### （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）
#### （b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果
#### （c）将(c)中的“向前5天”改为“向前非周末5天”，请再次计算结果

In [24]:
# （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）
df_new.rolling(window=50,min_periods=1).agg([np.mean, np.max])

销售额        
                   mean    amax
日期                             
2017-02-17  2154.000000  2154.0
2017-02-18  2124.500000  2154.0
2017-02-19  2569.333333  3459.0
2017-02-20  2476.500000  3459.0
2017-02-21  2463.800000  3459.0
...                 ...     ...
2019-11-09  2955.860000  4048.0
2019-11-10  2962.060000  4048.0
2019-11-11  2988.780000  4048.0
2019-11-12  2989.340000  4048.0
2019-11-13  2987.000000  4048.0

[1000 rows x 2 columns]

In [29]:
# （b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果【记为‘销售峰值’】
df_2018 = df_new.loc['2018']-df_new.loc['2018'].rolling(window=6,min_periods=-6).mean()
df_2018['销售峰值']=0
df_2018.loc[df_2018['销售额']>0,'销售峰值']=1
df_2018.head(20)


,销售额,销售峰值
日期,,
2018-01-01,0.000000,0
2018-01-02,-269.500000,0
2018-01-03,-210.333333,0
2018-01-04,-170.500000,0
2018-01-05,-145.200000,0
2018-01-06,339.000000,1
2018-01-07,-54.166667,0
2018-01-08,1021.833333,1
2018-01-09,-164.000000,0


In [27]:
# （c）将(b)中的“向前5天”改为“向前非周末5天”，请再次计算结果
df_2018_w = (df_new.loc['2018']-df_new.loc[pd.date_range(start='2018-1-1',end='2018-12-31',freq='B')].
             rolling(window=6,min_periods=-6).mean()).fillna(method='ffill')
df_2018_w['销售峰值']=0
df_2018_w.loc[df_2018['销售额']>0,'销售峰值']=1
df_2018_w.head(20)

,销售额,销售峰值
2018-01-01,0.000000,0
2018-01-02,-269.500000,0
2018-01-03,-210.333333,0
2018-01-04,-170.500000,0
2018-01-05,-145.200000,0
2018-01-06,-145.200000,1
2018-01-07,-145.200000,0
2018-01-08,1010.666667,1
2018-01-09,-77.666667,0
2018-01-10,90.333333,0
